### Loading the data

In [21]:
from llama_index.core import SimpleDirectoryReader

# Loading the documents
document = SimpleDirectoryReader("data").load_data()

### Chunking and Indexing the document

In [22]:
from llama_index.core import VectorStoreIndex, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Setting the embedding model
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

# Setting the text splitter
text_splitter = SentenceSplitter(chunk_size=500, chunk_overlap=10)
Settings.text_splitter = text_splitter


# Creating a vector index
index = VectorStoreIndex.from_documents(document, transformations=[text_splitter], embed_model=Settings.embed_model)

# Saving the index
index.storage_context.persist(persist_dir="storage")

### Setting up Gemini Client

In [29]:
import google.generativeai as genai
from dotenv import load_dotenv
import os 

load_dotenv()
api_key = os.getenv('api_key')

model = genai.GenerativeModel('gemini-2.0-flash')

genai.configure(api_key=api_key)

def retrieve_context(query, index):
    retriever = index.as_retriever()  # Create a retriever from the index
    relevant_docs = retriever.retrieve(query)  # Get the most relevant text chunks
    return "\n".join([doc.text for doc in relevant_docs]) 

